In [ ]:
import pandas as pd
import numpy as np
import ast
from typing import Tuple, List, Dict

def process_building_data(observer_file: str, building_file: str, tower_file: str, output_file: str) -> Tuple[List, List]:
    """
    Main function that processes building and observer data with separate logic for:
    1. Towers in the same building as observer
    2. Towers in different buildings
    
    Args:
        observer_file (str): Path to observer coordinates Excel file
        building_file (str): Path to building data Excel file
        tower_file (str): Path to tower data Excel file
        output_file (str): Path to save processed data
        
    Returns:
        Tuple containing:
        - List of building faces: Each face contains coordinates and metadata
        - List of observer faces: Each face contains coordinates and metadata
    """

    # 1. Observer Data Processing
    # Read the observer data file and get the tower number for the observer
    df_observer = pd.read_excel(observer_file)
    observer_tower = int(df_observer['tower_col'].iloc[0])
    
    # Create a nested dictionary to store observer coordinates
    # Structure: {tower_id: {flat_id: [coordinates]}}
    observer_coords_dict = {}
    for _, row in df_observer.iterrows():
        tower_id = row['tower_col']
        flat_id = row['series_col']
        # Convert coordinate values to integers and store as tuple
        coordinate = (int(row['x_col']), int(row['y_col']), int(row['z_col']))
        observer_coords_dict.setdefault(tower_id, {}).setdefault(flat_id, []).append(coordinate)
    
    # Flatten the nested dictionary into a list of coordinate lists
    observer_coordinates = [[observer_coords_dict[tower][flat] 
                           for flat in observer_coords_dict[tower]] 
                           for tower in observer_coords_dict]


    # 2. Building-Tower Relationship Mapping
    # Define the mapping between buildings and their corresponding towers
    buildings_map = {
        1: [1, 2],           # Building 1 contains towers 1 and 2
        2: [3, 4, 5],        # Building 2 contains towers 3, 4, and 5
        3: [6, 7, 8],        # Building 3 contains towers 6, 7, and 8
        4: [9, 10, 11],      # Building 4 contains towers 9, 10, and 11
        5: [12, 13, 14]      # Building 5 contains towers 12, 13, and 14
    }
    
    # Identify which building contains the observer tower
    observer_building = next(building for building, towers in buildings_map.items() 
                           if observer_tower in towers)

    # Separate towers into two categories:
    # a1. Towers in the same building as the observer
    same_building_towers = [tower for tower in buildings_map[observer_building] 
                          if tower != observer_tower]
   
    # a2. Towers in different buildings
    different_building_numbers = [building for building, towers in buildings_map.items() 
                                if building != observer_building]
    different_building_towers = [tower for building in different_building_numbers 
                               for tower in buildings_map[building]]
   
    # a3. Building and Tower Data Processing
    # Load building and tower data from Excel files
    df_building = pd.read_excel(building_file)
    df_tower = pd.read_excel(tower_file)
    
    # Process towers in the same building as observer
    same_building_data = df_tower[df_tower['Tower No'].isin(same_building_towers)].copy()
    same_building_data['Type'] = 'Same_Building_Tower'
    
    # diff_building_towers_data = df_tower[df_tower['Tower No'].isin(different_building_towers)].copy()
    # diff_building_towers_data['Type'] = 'Different_Building_Tower'
    
    # Process buildings different from observer's building
    diff_building_data = df_building[df_building['Building No'].isin(different_building_numbers)].copy()
    diff_building_data['Type'] = 'Different_Building'
    diff_building_data['Tower No'] = 0  # Set tower number to 0 for building-level data
    
    # Combine all processed data into a single DataFrame
    merged_data = pd.concat([
        same_building_data[['Building No', 'Tower No', 'x', 'y', 'z', 'Type']],
        #diff_building_towers_data[['Building No', 'Tower No', 'x', 'y', 'z', 'Type']],
        diff_building_data[['Building No', 'Tower No', 'x', 'y', 'z', 'Type']]
    ])
    
    # Save the processed data to Excel
    merged_data.to_excel(output_file, index=False)
    
    # a4. Face Generation
    building_faces = []
    
    # Process faces for towers in the same building
    same_building_groups = merged_data[merged_data['Type'] == 'Same_Building_Tower'].groupby(['Building No', 'Tower No'])
    for group_idx, (_, group) in enumerate(same_building_groups):
        # Create coordinate tuples from x, y, z columns
        coordinates = list(zip(group['x'], group['y'], group['z']))
        # Generate faces by taking groups of 4 coordinates
        for i in range(0, len(coordinates)-3, 2):
            face_coords = coordinates[i:i+4]
            # Create face tuple with metadata
            face = (*face_coords, observer_building, len(building_faces)+1, 'same_building')
            building_faces.append(face)
    
    # Process faces for different buildings
    diff_building_groups = merged_data[merged_data['Type'].isin(['Different_Building_Tower', 'Different_Building'])].groupby(['Building No', 'Tower No'])
    for group_idx, (_, group) in enumerate(diff_building_groups):
        coordinates = list(zip(group['x'], group['y'], group['z']))
        for i in range(0, len(coordinates)-3, 2):
            face_coords = coordinates[i:i+4]
            face = (*face_coords, group_idx+1, len(building_faces)+1, 'different_building')
            building_faces.append(face)
    
    # a5. Observer Face Generation
    observer_faces = []
    # Process observer coordinates into faces
    for m, tower in enumerate(observer_coordinates, observer_tower):
        for flat in tower:
            # Generate faces by taking groups of 4 coordinates
            for i in range(0, len(flat)-3, 2):
                face = (*flat[i:i+4], observer_building, len(observer_faces)+1, 'observer')
                observer_faces.append(face)
    
    return building_faces, observer_faces

In [131]:
# Main Execution Block
if __name__ == "__main__":
    # Define input and output file paths
    OBSERVER_FILE = r"C:\Users\Kalpesh\Downloads\NewApproach Sun Exposure\NewApproach Sun Exposure\Observer_towers\Observer_2.xlsx"
    BUILDING_FILE = r"C:\Users\Kalpesh\Downloads\NewApproach Sun Exposure (1)\NewApproach Sun Exposure\building_wise m3m.xlsx"
    TOWER_FILE = r"C:\Users\Kalpesh\Downloads\NewApproach Sun Exposure (1)\NewApproach Sun Exposure\TowerWise M3M.xlsx"
    OUTPUT_FILE = r"C:\Users\Kalpesh\Downloads\Jayesh Testing\analysis_results_eff\Tower_building_combinations\Tower2_building_final_M3M.xlsx"
    
    # Process all data in one go
    building_faces, observer_faces = process_building_data(
        OBSERVER_FILE,
        BUILDING_FILE,
        TOWER_FILE,
        OUTPUT_FILE
    )

1
[1]
[2, 3, 4, 5]
[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


In [109]:
import datetime  

latitude = 28.4595  # Latitude for the location (New Delhi, India)
longitude = 77.0266  # Longitude for the location (New Delhi, India)

time = datetime.datetime(2023, 7, 15, 14, 0, 0)  # July 15, 2023, at 14:00 (2 PM)

sun_azimuth_morning = 67.7756271507185  # Sun azimuth in the morning
sun_azimuth_afternoon = 228.2852958  # Sun azimuth at 1 PM
sun_azimuth_evening = 309.08750082861  # Sun azimuth in the evening

sun_elevation_morning = 4.557596553  # Sun elevation in the morning
sun_elevation_afternoon = 79.96185006  # Sun elevation at 1 PM
sun_elevation_evening = -19.23511277  # Sun elevation in the evening (below the horizon)


In [110]:
def wall_orientation(cross_product1, cross_product2):
    """Determine wall orientation based on cross products."""
    if cross_product1 < 0 and cross_product2 < 0:
        return "Front"  
    elif cross_product1 > 0 and cross_product2 > 0:
        return "Back"  
    else:
        return "Parallel"  

def calculate_cross_product(observer_face, building_face):
    """Calculate cross products to determine the orientation of walls."""
    # Extract points from the observer's face at ground level (z=0)
    observer_point1, observer_point2 = [point for point in observer_face[:4] if point[2] == 0]
    
    # Extract points from the building's face at ground level (z=0)
    building_point1, building_point2 = [point for point in building_face[:4] if point[2] == 0]

    # Calculate vector ab from the observer's two points
    ab = np.array(observer_point2) - np.array(observer_point1)
    w1 = np.array(building_point1) - np.array(observer_point1)  # Vector to the first building point
    w2 = np.array(building_point2) - np.array(observer_point1)  # Vector to the second building point
    
    # Calculate cross products to determine relative positions
    cross_product_result = np.cross(ab, w1).astype(int)  # Cross product with first building vector
    cross_product_result1 = np.cross(ab, w2).astype(int)  # Cross product with second building vector
    
    return cross_product_result, cross_product_result1  

def check_azimuth_overlap(min_obstacle, max_obstacle, range_start, range_end):
    """Check if azimuth ranges overlap."""
    # Handle cases where the range wraps around 360 degrees
    if range_start > range_end:
        return not (max_obstacle < range_start and min_obstacle > range_end)  # Check for overlap
    return not (min_obstacle > range_end or max_obstacle < range_start)  # Check for overlap in standard range

def check_buildings(building_x1, building_y1, building_x2, building_y2,
                    observer_x, observer_y,
                    sun_azimuth_morning, sun_azimuth_evening):
    """Check if a building wall can cast a shadow based on sun position and observer location."""
    # Calculate the azimuth angles of the building points relative to the observer
    azimuth_obstacle1 = np.degrees(np.arctan2(building_x1 - observer_x, building_y1 - observer_y)) % 360  
    azimuth_obstacle2 = np.degrees(np.arctan2(building_x2 - observer_x, building_y2 - observer_y)) % 360   
    
    # Calculate the relative position of the building points to the observer
    ab_aw1_z = building_x1 - observer_x  
    ab_aw2_z = building_x2 - observer_x  # Cross product = [obst_x - obsv_x]*1 - [obst_y - obsv_y]*0
    
    # Determine the orientation of the building wall
    orientation = wall_orientation(ab_aw1_z, ab_aw2_z)
    sun_range_crosses_zero = sun_azimuth_morning > sun_azimuth_evening  # Check if the sun path crosses zero degrees
    
    # Determine minimum and maximum azimuth angles of the obstacles
    min_obstacle = min(azimuth_obstacle1, azimuth_obstacle2)
    max_obstacle = max(azimuth_obstacle1, azimuth_obstacle2)
    
    if orientation in ["Front", "Back"]:  # If the wall is in front or back of the observer
        if sun_range_crosses_zero:  # If the sun path wraps around
            morning_range = (sun_azimuth_morning, 360)  # Morning azimuth range
            evening_range = (0, sun_azimuth_evening)  # Evening azimuth range
            # Check for overlaps with both morning and evening ranges
            overlaps_morning = check_azimuth_overlap(min_obstacle, max_obstacle, *morning_range)
            overlaps_evening = check_azimuth_overlap(min_obstacle, max_obstacle, *evening_range)
            return (overlaps_morning or overlaps_evening)  # Return true if any overlap is found
        else:
            # Check for overlap in the normal azimuth range
            return check_azimuth_overlap(min_obstacle, max_obstacle, sun_azimuth_morning, sun_azimuth_evening)
    else:  # If the wall is parallel to the observer's view
        # Define a function to check if a point could cast a shadow based on azimuth
        def point_could_cast_shadow(azimuth):
            return (azimuth > sun_azimuth_morning and azimuth < sun_azimuth_evening) if not sun_range_crosses_zero else \
                   (azimuth > sun_azimuth_morning or azimuth < sun_azimuth_evening)

        # Check if either building point could cast a shadow
        return (point_could_cast_shadow(azimuth_obstacle1) or point_could_cast_shadow(azimuth_obstacle2))


In [111]:
import matplotlib.pyplot as plt

def check_shadow_on_observer_wall(self, building_data, observer_data, 
                                   sun_azimuth_morning, sun_azimuth_evening):
    """Calculate shadow information for plotting."""
    shadow_checks = []
    
    # Check each point of the observer wall
    for obs_idx in range(len(observer_data)):
        obs_x = observer_data['x_col'].iloc[obs_idx]
        obs_y = observer_data['y_col'].iloc[obs_idx]
        
        # Check against each segment of the obstacle wall
        shadow_status = self.check_buildings(
            building_data['x_col'].iloc[0], building_data['y_col'].iloc[0],  # First point
            building_data['x_col'].iloc[1], building_data['y_col'].iloc[1],  # Second point
            obs_x, obs_y,
            sun_azimuth_morning, sun_azimuth_evening
        )
        shadow_checks.append(shadow_status)
    
    # Calculate coverage
    shadow_coverage = sum(shadow_checks) / len(shadow_checks) if shadow_checks else 0
    
    return {
        'has_shadow': any(shadow_checks),
        'full_shadow': all(shadow_checks),
        'coverage': shadow_coverage
    }

def generate_plot_filename(self, plot_type, observer_tower, observer_wall, building_no=None, plot_index=None):
    """Generate standardized plot filename with unique identifiers."""
    base_name = f"{plot_type}_analysis_tower{observer_tower}_wall{observer_wall}"
    # Append building number and index for shadow plots; index only for orientation plots
    return f"{base_name}_building{building_no}_idx{plot_index}.png" if plot_type == "shadow" else f"{base_name}_idx{plot_index}.png"

def plot_shadow_analysis(self, building_data, observer_data, plot_index):
    """Plot shadow analysis with coordinates and shadow status."""
    observer_subsets = [observer_data.iloc[[i, i+1]] for i in range(len(observer_data) - 1)]
    wall_colors = ['blue', 'red', 'green', 'purple', 'orange', 'brown', 'pink', 'gray']
    
    for idx, observer_subset in enumerate(observer_subsets):
        plt.figure(figsize=(12, 10))
        
        # Extract observer and building information for filename
        observer_tower = observer_subset['tower_col'].iloc[0]
        observer_wall = observer_subset['wall_no'].iloc[0]
        building_no = building_data['building_no'].iloc[0] if 'building_no' in building_data.columns else 'unknown'
        
        # Generate unique filename with plot index
        filename = self.generate_plot_filename("shadow", observer_tower, observer_wall, building_no, plot_index)
        
        # Plot observer wall with coordinates
        plt.plot(observer_subset['x_col'], observer_subset['y_col'], marker='o', color='black', linewidth=2, label=f'Observer Wall {idx}')
        
        # Annotate observer coordinates
        for x, y in zip(observer_subset['x_col'], observer_subset['y_col']):
            plt.annotate(f'({x}, {y})', (x, y), xytext=(5, 5), textcoords='offset points', fontsize=8,
                         bbox=dict(facecolor='white', edgecolor='none', alpha=0.7))
        
        # Process obstacle walls by grouping points
        tower_points = building_data[building_data['tower_col'] == 1]
        wall_segments = [(tower_points['x_col'].iloc[i:i+2].tolist(), tower_points['y_col'].iloc[i:i+2].tolist())
                         for i in range(0, len(tower_points), 2)]
        
        # Plot obstacle walls with coordinates and shadow status
        for wall_idx, (x_coords, y_coords) in enumerate(wall_segments):
            current_building = pd.DataFrame({'x_col': x_coords, 'y_col': y_coords})
            
            # Calculate shadow information
            shadow_info = self.check_shadow_on_observer_wall(current_building, observer_subset, self.sun_azimuth_morning, self.sun_azimuth_evening)
            
            # Plot wall
            plt.plot(x_coords, y_coords, color=wall_colors[wall_idx % len(wall_colors)], linewidth=2, marker='o', label=f'Obstacle Wall {wall_idx + 1}')
            
            # Annotate obstacle points
            for x, y in zip(x_coords, y_coords):
                plt.annotate(f'({x}, {y})', (x, y), xytext=(5, 5), textcoords='offset points', fontsize=8,
                             bbox=dict(facecolor='white', edgecolor='none', alpha=0.7))
            
            # Add shadow status text
            midpoint_x, midpoint_y = sum(x_coords) / len(x_coords), sum(y_coords) / len(y_coords)
            if shadow_info['full_shadow']:
                status_text = "Full Shadow"
                color = 'red'
            elif shadow_info['has_shadow']:
                status_text = f"Partial\n{shadow_info['coverage'] * 100:.1f}%"
                color = 'orange'
            else:
                status_text = "No Shadow"
                color = 'green'
                
            plt.text(midpoint_x, midpoint_y, f'Wall {wall_idx + 1}\n{status_text}', ha='center', va='center',
                     bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'), color=color, fontsize=10)
        
        # Add sun path information
        plt.text(0.02, 0.98, f'Sun Path:\nMorning: {self.sun_azimuth_morning:.1f}°\nEvening: {self.sun_azimuth_evening:.1f}°',
                 transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.7), verticalalignment='top')
        
        plt.title(f'Shadow Analysis for Observer Wall {idx} (Plot {plot_index})')
        plt.xlabel('X Coordinate')
        plt.ylabel('Y Coordinate')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True)
        plt.tight_layout()
        
        # Save plot with unique filename
        try:
            save_path = self.shadow_plots / filename
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            print(f"Successfully saved shadow plot: {save_path}")
        except Exception as e:
            print(f"Error saving shadow plot {filename}: {str(e)}")
        finally:
            plt.close()

def plot_wall_orientations(self, building_data, observer_data, observer_subsets, plot_index):
    """Plot obstacle and observer walls with orientations and coordinates."""
    for idx, observer_subset in enumerate(observer_subsets):
        plt.figure(figsize=(12, 10))
        
        # Get observer information for filename
        observer_tower = observer_subset['tower_col'].iloc[0]
        observer_wall = observer_subset['wall_no'].iloc[0]
        
        # Generate unique filename with plot index
        filename = self.generate_plot_filename("orientation", observer_tower, observer_wall, plot_index=plot_index)
        
        # Plot obstacle walls with coordinates
        for tower_num in building_data['tower_col'].unique():
            tower_points = building_data[building_data['tower_col'] == tower_num]
            plt.plot(tower_points['x_col'], tower_points['y_col'], marker='o', label=f'Obstacle Tower {tower_num}')
            
            # Annotate obstacle points
            for x, y in zip(tower_points['x_col'], tower_points['y_col']):
                plt.annotate(f'({x}, {y})', (x, y), xytext=(5, 5), textcoords='offset points', fontsize=8,
                             bbox=dict(facecolor='white', edgecolor='none', alpha=0.7))
        
        # Plot all observer walls
        plt.plot(observer_data['x_col'], observer_data['y_col'], marker='o', color='lightblue', label='All Observer Walls')
        
        # Highlight current observer wall
        plt.plot(observer_subset['x_col'], observer_subset['y_col'], marker='o', color='black', linewidth=2,
                 label=f'Highlighted Observer Wall {idx}')
        
        # Annotate observer points
        for x, y in zip(observer_subset['x_col'], observer_subset['y_col']):
            plt.annotate(f'({x}, {y})', (x, y), xytext=(5, 5), textcoords='offset points', fontsize=8,
                         bbox=dict(facecolor='white', edgecolor='none', alpha=0.7))
        
        plt.xlabel('West-East')
        plt.ylabel('South-North')
        plt.title(f'Plot {plot_index}: Obstacle and Observer Walls with Orientations')
        plt.grid(True)
        plt.legend(loc='lower right')
        
        # Save plot with unique filename
        try:
            save_path = self.orientation_plots / filename
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            print(f"Successfully saved orientation plot: {save_path}")
        except Exception as e:
            print(f"Error saving orientation plot {filename}: {str(e)}")
        finally:
            plt.close()

In [ ]:
# Initialize lists to store observer walls, building walls, and combined results
observer_walls = []
building_walls = []
observer_building_combine = []

# Loop through each observer wall
for i, observer_wall in enumerate(observer_faces):
    print('Observer_wall', observer_wall)
    
    # Loop through each building wall
    for j, building_wall in enumerate(building_faces):
        print('Building_wall:', building_wall)
        
        # Extract coordinates for the observer wall
        observer_wall_x1 = observer_wall[0][0]
        observer_wall_y1 = observer_wall[0][1]
        observer_wall_x2 = observer_wall[2][0]
        observer_wall_y2 = observer_wall[2][1]
        
        # Extract coordinates for the building wall
        building_wall_x1 = building_wall[0][0]
        building_wall_y1 = building_wall[0][1]
        building_wall_x2 = building_wall[2][0]
        building_wall_y2 = building_wall[2][1]
        
        # Extract additional information about the observer and building
        observer_tower = observer_wall[4]
        building_tower = building_wall[4]
        building_no = building_wall[5]

        # Check if the building casts a shadow on the observer wall
        building_status = check_buildings(
            building_wall_x1, building_wall_y1, 
            building_wall_x2, building_wall_y2, 
            observer_wall_y1, observer_wall_y2, 
            sun_azimuth_morning, sun_azimuth_evening
        )
        
        # Append observer and building walls to their respective lists
        observer_walls.append(observer_wall)
        building_walls.append(building_wall)

        # Calculate the cross product between observer and building walls
        cross_product_result = calculate_cross_product(observer_wall[:5], building_wall[:5])
        
        # Combine relevant information into a single tuple and append to results list
        observer_building_combine.append((
            observer_wall[4],  # Observer tower number
            observer_wall[5],  # Observer wall number
            observer_wall[0:4],  # Coordinates of the observer wall
            building_no,  # Building number
            building_tower,  # Building tower number
            building_wall[5],  # Building wall number
            building_wall[0:4],  # Coordinates of the building wall
            building_status,  # Status of shadow casting
            cross_product_result[0][2],  # z-component of the first cross product
            cross_product_result[1][2]   # z-component of the second cross product
        ))

# Create DataFrames from the collected data
observer_walls_df = pd.DataFrame(observer_walls, columns=['p1', 'p2', 'p3', 'p4', 'tower', 'face', 'Type'])
building_walls_df = pd.DataFrame(building_walls, columns=['p1', 'p2', 'p3', 'p4', 'tower', 'face', 'Type'])

# Combine results into a DataFrame for easier analysis
results_combine_df = pd.DataFrame(observer_building_combine, columns=[
    'Tower_number', 'Observer_wall_no', 'observer_walls', 
    'Building No', 'tower_no', 'Building_wall_no', 
    'Building_walls', 'Building_status', 
    'ab*aw1_z', 'ab*aw2_z'
])

# Determine wall orientation based on the cross product results
results_combine_df['wall_orientation'] = results_combine_df.apply(
    lambda x: wall_orientation(x['ab*aw1_z'], x['ab*aw2_z']), axis=1
)

SyntaxError: invalid syntax (3805992392.py, line 29)

In [113]:
with pd.ExcelWriter(r"C:\Users\Kalpesh\Downloads\Jayesh Testing\analysis_results_eff\shadow_analysis_results.xlsx") as writer:
    observer_walls_df.to_excel(writer, sheet_name='Observer Walls', index=False)
    building_walls_df.to_excel(writer, sheet_name='Building Walls', index=False)
    results_combine_df.to_excel(writer,sheet_name='Observer_building_key',index=False)


In [94]:
results_combine_df1 = results_combine_df[(results_combine_df['wall_orientation']!='Back') & (results_combine_df['Building_status']==True)]                  
# Consider only those rows for further calculations which do not have wall_orientation as 'Back' and building_status as True
results_combine_df1.reset_index(drop=True,inplace=True)

In [95]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import math
import pytz
import pysolar as ps
import pandas as pd  # Import pandas for handling Excel files
import os

def calculate_Ow(observer_wall_x, observer_wall_y, observer_wall_z, building_wall_x, building_wall_y, building_wall_z):
    """Calculate the angle of obstruction (Ow) between the observer wall and building wall."""
    
    # Calculate the squared distance between the observer and building wall
    squared_distance = (building_wall_y - observer_wall_y) ** 2 + (building_wall_x - observer_wall_x) ** 2
    if squared_distance < 0:
        raise ValueError("Error: Squared distance is negative.")
    
    distance = math.sqrt(squared_distance)  # Calculate Euclidean distance

    # Calculate the height difference between the two walls
    height_diff = building_wall_z - observer_wall_z

    # Calculate the angle Ow (angle of obstruction)
    Ow = 0 if distance == 0 else math.degrees(math.atan(height_diff / distance))
    
    return Ow

def get_sun_data(latitude, longitude, date_time):
    """Retrieve sun elevation and azimuth based on the given latitude, longitude, and time."""
    
    # Set timezone for the specified location
    timezone = pytz.timezone('Asia/Kolkata')
    date_time_local = timezone.localize(date_time)  # Localize the datetime to the specified timezone

    # Calculate sun position using pysolar
    sun_elevation = ps.solar.get_altitude(latitude, longitude, date_time_local)
    sun_azimuth = ps.solar.get_azimuth(latitude, longitude, date_time_local)

    return sun_elevation, sun_azimuth

def get_shadow_status_and_elevation(observer_wall1_x, observer_wall1_y, observer_wall1_z, observer_wall2_x, observer_wall2_y, observer_wall2_z, 
                                     building_wall1_x, building_wall1_y, building_wall1_z, building_wall2_x, building_wall2_y, building_wall2_z, 
                                     latitude, longitude, time):
    """Determine shadow status for two observer walls based on building walls and sun position."""
    
    # Calculate Ow for the two observer walls against the two building walls
    obs1_bldg1 = calculate_Ow(observer_wall1_x, observer_wall1_y, observer_wall1_z, building_wall1_x, building_wall1_y, building_wall1_z)
    obs1_bldg2 = calculate_Ow(observer_wall1_x, observer_wall1_y, observer_wall1_z, building_wall2_x, building_wall2_y, building_wall2_z)
    obs2_bldg1 = calculate_Ow(observer_wall2_x, observer_wall2_y, observer_wall2_z, building_wall1_x, building_wall1_y, building_wall1_z)
    obs2_bldg2 = calculate_Ow(observer_wall2_x, observer_wall2_y, observer_wall2_z, building_wall2_x, building_wall2_y, building_wall2_z)

    # Get sun data for the current time
    elevation, azimuth = get_sun_data(latitude, longitude, time)

    # Determine shadow status based on the elevation of the sun relative to the maximum Ow
    shadow_status_obs1 = elevation <= max(obs1_bldg1, obs1_bldg2)
    shadow_status_obs2 = elevation <= max(obs2_bldg1, obs2_bldg2)

    return shadow_status_obs1, shadow_status_obs2, elevation, obs1_bldg1, obs1_bldg2, obs2_bldg1, obs2_bldg2

def get_closest_times(elevation, excel_file):
    """Find the closest times in an Excel file to a given sun elevation."""
    
    # Read data from Excel file
    df = pd.read_excel(excel_file)
    df['Time'] = pd.to_datetime(df['Time'])  # Convert time column to datetime

    # Split DataFrame into two based on time (before and after noon)
    df_before_12 = df[df['Time'].dt.hour < 12]
    df_after_12 = df[df['Time'].dt.hour >= 12]

    # Find closest elevation times before noon
    closest_elevation_row_before_12 = df_before_12.iloc[(df_before_12['Elevation'] - elevation).abs().argsort()[:1]]
    closest_time_before_12 = pd.Timestamp(closest_elevation_row_before_12['Time'].values[0]).strftime('%H')

    # Find closest elevation times after noon
    closest_elevation_row_after_12 = df_after_12.iloc[(df_after_12['Elevation'] - elevation).abs().argsort()[:1]]
    closest_time_after_12 = pd.Timestamp(closest_elevation_row_after_12['Time'].values[0]).strftime('%H')

    return closest_time_before_12, closest_time_after_12

In [ ]:
import datetime
import numpy as np  # Use numpy for efficient numerical operations
import pandas as pd  # Import pandas for DataFrame handling

# Path to the Excel file containing sun data
excel_file_path = r"C:\Users\Kalpesh\Downloads\NewApproach Sun Exposure\NewApproach Sun Exposure\Azimuth_elevation\Sun_data_Pysolar_new.xlsx"

# Initialize lists to store results before the loop
obs_x1_list, obs_y1_list, obs_z1_list = [], [], []
obs_x2_list, obs_y2_list, obs_z2_list = [], [], []
building_x1_list, building_y1_list, building_z1_list = [], [], []
building_x2_list, building_y2_list, building_z2_list = [], [], []
shadow_status_obs1_list, shadow_status_obs2_list = [], []
elevation_list = []
closest_time_before_12, closest_time_after_12 = [], []

# Iterate over each row in the results DataFrame
for index, row in results_combine_df1.iterrows():
    #print('Row:', row)

    # Extract observer and building wall coordinates
    observer_walls = row['observer_walls']
    building_walls = row['Building_walls']
    
    obs_x1, obs_y1, obs_z1 = observer_walls[0][0], observer_walls[0][1], observer_walls[0][2] 
    obs_x2, obs_y2, obs_z2 = observer_walls[2][0], observer_walls[2][1], observer_walls[2][2]  # Third vertex of the observer wall
    
    building_x1, building_y1, building_z1 = building_walls[1][0], building_walls[1][1], building_walls[1][2]   # First vertex of the building wall
    building_x2, building_y2, building_z2 = building_walls[3][0], building_walls[3][1], building_walls[3][2]  # Second vertex of the building wall

    # Calculate shadow status and elevation angles
    shadow_status_obs1, shadow_status_obs2, elevation, t1_angle_elevation, t2_angle_elevation, t3_angle_elevation, t4_angle_elevation = get_shadow_status_and_elevation(
        obs_x1, obs_y1, obs_z1, obs_x2, obs_y2, obs_z2,
        building_x1, building_y1, building_z1, building_x2, building_y2, building_z2,
        latitude, longitude, time
    )

    # Update the results DataFrame with elevation angles
    results_combine_df1.at[index, 't1_elevation'] = t1_angle_elevation
    results_combine_df1.at[index, 't2_elevation'] = t2_angle_elevation
    results_combine_df1.at[index, 't3_elevation'] = t3_angle_elevation
    results_combine_df1.at[index, 't4_elevation'] = t4_angle_elevation
    
    # Calculate the maximum elevation across the four angles
    results_combine_df1['max_elevation'] = results_combine_df1[['t1_elevation', 't2_elevation', 't3_elevation', 't4_elevation']].max(axis=1)
    max_elevation = results_combine_df1['max_elevation']

    # Get closest times for maximum elevation
    closest_time_before_12, closest_time_after_12 = get_closest_times(max_elevation, excel_file_path)
    results_combine_df1.at[index, 'Min_elevation_time'] = closest_time_before_12
    results_combine_df1.at[index, 'Max_elevation_time'] = closest_time_after_12

    # Append values to respective lists for later analysis or output
    shadow_status_obs1_list.append(shadow_status_obs1)
    shadow_status_obs2_list.append(shadow_status_obs2)
    elevation_list.append(elevation)

# Assign collected lists to DataFrame columns for further analysis
results_combine_df1['shadow_status_obs1'] = shadow_status_obs1_list
results_combine_df1['shadow_status_obs2'] = shadow_status_obs2_list
results_combine_df1['elevation'] = elevation_list

In [ ]:
excel_file = r"C:\Users\Kalpesh\Downloads\NewApproach Sun Exposure\NewApproach Sun Exposure\Azimuth_elevation\Sun_data_Pysolar_new.xlsx"
df = pd.read_excel(excel_file)
for index, row in results_combine_df1.iterrows():
    results_combine_df1['max_elevation'] = np.max(results_combine_df1[['t1_elevation', 't2_elevation', 't3_elevation', 't4_elevation']], axis=1)
    max_elevation=row['max_elevation']
    
    closest_time_before_12, closest_time_after_12 = get_closest_times(max_elevation, excel_file_path)
    results_combine_df1.at[index,'Min_elevation_time']=int(closest_time_before_12)
    results_combine_df1.at[index,'Max_elevation_time']=int(closest_time_after_12)

    #print("row:-",row)


In [98]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
import math
import pytz
import pysolar.solar as ps

def calculate_azimuth_ccw_or_cw(x_observer, y_observer, x_target, y_target, is_ccw=False):
    """Calculate azimuth angle with respect to positive Y axis"""
    if (x_target == x_observer):
        return 0
    angle = np.degrees(np.arctan2(y_target - y_observer, x_target - x_observer)) % 360
    if is_ccw:
        return -angle + 90
    return 90 - angle

def get_sun_data(latitude, longitude, time):
    """Get sun elevation and azimuth using pysolar"""
    date = time
    timezone = pytz.timezone('Asia/Kolkata')
    date = timezone.localize(date)
    
    altitude = ps.get_altitude(latitude, longitude, date)
    azimuth = ps.get_azimuth(latitude, longitude, date)
    
    return altitude, azimuth

def get_shadow_status_and_azimuth(obs_x1, obs_y1, obs_x2, obs_y2, 
                                 building_x1, building_y1, building_x2, building_y2,
                                 latitude, longitude, time):
    """Calculate shadow status and azimuth angles for both observers"""
    
    # Calculate angles for observer 1
    t1_angle = calculate_azimuth_ccw_or_cw(obs_x1, obs_y1, building_x1, building_y1)
    t2_angle = calculate_azimuth_ccw_or_cw(obs_x1, obs_y1, building_x2, building_y2)
    
    # Calculate angles for observer 2
    t3_angle = calculate_azimuth_ccw_or_cw(obs_x2, obs_y2, building_x1, building_y1)
    t4_angle = calculate_azimuth_ccw_or_cw(obs_x2, obs_y2, building_x2, building_y2)
    
    # Get sun data
    elevation, azimuth = get_sun_data(latitude, longitude, time)
    
    # Calculate cross products for both observers
    cross_prod_obs1_b1 = building_x1 - obs_x1               # Cross product = [obst_x - obsv_x]*1 - [obst_y - obsv_y]*0
    cross_prod_obs1_b2 = building_x2 - obs_x1               
    cross_prod_obs2_b1 = building_x1 - obs_x2 
    cross_prod_obs2_b2 = building_x2 - obs_x2 
    
    # Determine shadow status for observer 1
    shadow_status_1 = False
    if (cross_prod_obs1_b1 * cross_prod_obs1_b2 < 0):
        if t1_angle <= azimuth <= t2_angle or t1_angle >= azimuth >= t2_angle:
            shadow_status_1 = True
            
    # Determine shadow status for observer 2
    shadow_status_2 = False
    if (cross_prod_obs2_b1 * cross_prod_obs2_b2 < 0):
        if t3_angle <= azimuth <= t4_angle or t3_angle >= azimuth >= t4_angle:
            shadow_status_2 = True
    
    return shadow_status_1, shadow_status_2, azimuth, t1_angle, t2_angle, t3_angle, t4_angle

def plot_shadow_analysis(obs_x1, obs_y1, obs_x2, obs_y2, 
                        building_x1, building_y1, building_x2, building_y2,
                        azimuth, t1_angle, t2_angle, t3_angle, t4_angle,
                        shadow_status_1, shadow_status_2, save_dir, index, building_id=None):
    """
    Create and save plot for shadow analysis with unique naming
    
    Parameters:
    -----------
    ... (previous parameters remain same)
    index : int
        Current row index from DataFrame
    building_id : str, optional
        Unique identifier for the building being analyzed
    """
    plt.figure(figsize=(12, 10))
    
    # Plot observers
    plt.plot(obs_x1, obs_y1, 'go', label='Observer 1')
    plt.plot(obs_x2, obs_y2, 'go', label='Observer 2')
    
    # Plot building points
    plt.plot(building_x1, building_y1, 'ro', label='Building Point 1')
    plt.plot(building_x2, building_y2, 'ro', label='Building Point 2')
    
    # Plot building wall
    plt.plot([building_x1, building_x2], [building_y1, building_y2], 'b--', label='Building Wall')
    
    # Plot lines from observers to building points
    plt.plot([obs_x1, building_x1], [obs_y1, building_y1], 'k-', 
             label=f'Obs 1 to Point 1 (Azimuth: {t1_angle:.2f}°)')
    plt.plot([obs_x1, building_x2], [obs_y1, building_y2], 'k-', 
             label=f'Obs 1 to Point 2 (Azimuth: {t2_angle:.2f}°)')
    plt.plot([obs_x2, building_x1], [obs_y2, building_y1], 'r-', 
             label=f'Obs 2 to Point 1 (Azimuth: {t3_angle:.2f}°)')
    plt.plot([obs_x2, building_x2], [obs_y2, building_y2], 'r-', 
             label=f'Obs 2 to Point 2 (Azimuth: {t4_angle:.2f}°)')
    
    # Plot sun direction arrows
    plt.quiver(obs_x1, obs_y1, np.sin(np.radians(sun_azimuth_morning)), np.cos(np.radians(sun_azimuth_morning)),
               scale=10, color='#f96343', label=f'Sun Azimuth Morning : {sun_azimuth_morning:.2f}°', angles='xy')
    plt.quiver(obs_x1, obs_y1, np.sin(np.radians(sun_azimuth_afternoon)), np.cos(np.radians(sun_azimuth_afternoon)),
               scale=10, color='#d3de50', label=f'Sun Azimuth Afternoon : {sun_azimuth_afternoon:.2f}°', angles='xy')
    plt.quiver(obs_x1, obs_y1, np.sin(np.radians(sun_azimuth_evening)), np.cos(np.radians(sun_azimuth_evening)),
               scale=10, color='#df684e', label=f'Sun Azimuth Evening : {sun_azimuth_evening:.2f}°', angles='xy')
    plt.quiver(obs_x2, obs_y2, np.sin(np.radians(sun_azimuth_morning)), np.cos(np.radians(sun_azimuth_morning)),
               scale=10, color='#f96343')
    plt.quiver(obs_x2, obs_y2, np.sin(np.radians(sun_azimuth_afternoon)), np.cos(np.radians(sun_azimuth_afternoon)),
               scale=10, color='#d3de50')
    plt.quiver(obs_x2, obs_y2, np.sin(np.radians(sun_azimuth_evening)), np.cos(np.radians(sun_azimuth_evening)),
               scale=10, color='#df684e')
    
    # Add shadow status
    plt.text(0.05, 0.95, f'Observer 1 Shadow: {shadow_status_1}', 
             transform=plt.gca().transAxes, fontsize=12, color='red')
    plt.text(0.05, 0.90, f'Observer 2 Shadow: {shadow_status_2}', 
             transform=plt.gca().transAxes, fontsize=12, color='red')
    
    plt.xlabel('West-East')
    plt.ylabel('South-North')
    
    # Modified title to include more information
    title = f'Shadow Analysis - Building {building_id if building_id else index}'
    if shadow_status_1 or shadow_status_2:
        title += ' (Shadow Present)'
    plt.title(title)
    
    plt.grid(True)
    plt.legend()
    plt.axis('equal')
    
    # Create unique filename
    filename_parts = [
        f'building_{building_id if building_id else index}',
        f'obs1_{obs_x1:.1f}_{obs_y1:.1f}',
        f'obs2_{obs_x2:.1f}_{obs_y2:.1f}',
        f'azimuth_{azimuth:.1f}'
    ]
    if shadow_status_1 or shadow_status_2:
        filename_parts.append('shadow')
    
    plot_filename = '_'.join(filename_parts) + '.png'
    
    # Save plot
    os.makedirs(save_dir, exist_ok=True)
    plt.savefig(os.path.join(save_dir, plot_filename))
    plt.close()

def get_time_from_azimuth(azimuth, data):
    """Find closest time for given azimuth from data"""
    closest_row = min(data.iterrows(), key=lambda x: abs(x[1]['Azimuth'] - azimuth))[1]
    return closest_row['Time'].hour

In [99]:
# Your existing imports and DataFrame setup remains the same

save_dir = r"C:\Users\Kalpesh\Downloads\Jayesh Testing\analysis_results_eff\plots"  # Replace with your desired save path

# Initialize lists before the loop
shadow_status_1_list = []
shadow_status_2_list = []
azimuth_list = []
ob1_b1 = []

# Load sun data
excel_file = r"C:\Users\Kalpesh\Downloads\NewApproach Sun Exposure\NewApproach Sun Exposure\Azimuth_elevation\Sun_data_Pysolar_new.xlsx"
df = pd.read_excel(excel_file)

for index, row in results_combine_df1.iterrows():
    observer_walls = row['observer_walls']
    building_walls = row['Building_walls']
    
    # Extract coordinates
    obs_x1, obs_y1, obs_z1 = observer_walls[0][0], observer_walls[0][1], observer_walls[0][2] 
    obs_x2, obs_y2, obs_z2 = observer_walls[2][0], observer_walls[2][1], observer_walls[2][2] 
    
    building_x1, building_y1, building_z1 = building_walls[1][0], building_walls[1][1], building_walls[1][2]   
    building_x2, building_y2, building_z2 = building_walls[3][0], building_walls[3][1], building_walls[3][2]

    # Get building ID if available in your DataFrame
    building_id = row.get('building_id', None)  # Replace 'building_id' with your actual column name
    
    shadow_status_1, shadow_status_2, azimuth, t1_angle, t2_angle, t3_angle, t4_angle = get_shadow_status_and_azimuth(
        obs_x1, obs_y1, obs_x2, obs_y2,
        building_x1, building_y1, building_x2, building_y2,
        latitude, longitude, time
    )
    
    # Store results and create plot with unique naming
    results_combine_df1.at[index, 't1_azimuth'] = t1_angle
    results_combine_df1.at[index, 't2_azimuth'] = t2_angle
    results_combine_df1.at[index, 't3_azimuth'] = t3_angle
    results_combine_df1.at[index, 't4_azimuth'] = t4_angle
    
    # Create plot with unique naming
    plot_shadow_analysis(
        obs_x1, obs_y1, obs_x2, obs_y2,
        building_x1, building_y1, building_x2, building_y2,
        azimuth, t1_angle, t2_angle, t3_angle, t4_angle,
        shadow_status_1, shadow_status_2,
        save_dir, index, building_id
    )

# Calculate min/max azimuth and corresponding times
results_combine_df1['Min_azimuth'] = np.min(results_combine_df1[['t1_azimuth', 't2_azimuth', 't3_azimuth', 't4_azimuth']], axis=1)
results_combine_df1['Max_azimuth'] = np.max(results_combine_df1[['t1_azimuth', 't2_azimuth', 't3_azimuth', 't4_azimuth']], axis=1)


for index, row in results_combine_df1.iterrows():
    min_azimuth = row['Min_azimuth']
    max_azimuth = row['Max_azimuth']

    min_time = get_time_from_azimuth(min_azimuth, df)
    max_time = get_time_from_azimuth(max_azimuth, df)
   
    results_combine_df1.at[index, 'Min_azimuth_time'] = min_time
    results_combine_df1.at[index, 'Max_azimuth_time'] = max_time

C:\Users\Kalpesh\AppData\Local\Temp\ipykernel_22168\809298478.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_combine_df1.at[index, 't1_azimuth'] = t1_angle
C:\Users\Kalpesh\AppData\Local\Temp\ipykernel_22168\809298478.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_combine_df1.at[index, 't2_azimuth'] = t2_angle
C:\Users\Kalpesh\AppData\Local\Temp\ipykernel_22168\809298478.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [100]:
# Convert list-like columns to tuples
results_combine_df['observer_walls'] = results_combine_df['observer_walls'].apply(tuple)
results_combine_df1['observer_walls'] = results_combine_df1['observer_walls'].apply(tuple)

results_combine_df['Building_walls'] = results_combine_df['Building_walls'].apply(tuple)
results_combine_df1['Building_walls'] = results_combine_df1['Building_walls'].apply(tuple)

# Perform the merge
final = pd.merge(
    results_combine_df,
    results_combine_df1,
    on=[
        'Tower_number', 'Observer_wall_no', 'observer_walls',
        'Building No', 'tower_no', 'Building_wall_no',
        'Building_walls', 'Building_status', 'ab*aw1_z', 
        'ab*aw2_z', 'wall_orientation'
    ],
    how='left'
)

C:\Users\Kalpesh\AppData\Local\Temp\ipykernel_22168\1537797534.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_combine_df1['observer_walls'] = results_combine_df1['observer_walls'].apply(tuple)
C:\Users\Kalpesh\AppData\Local\Temp\ipykernel_22168\1537797534.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_combine_df1['Building_walls'] = results_combine_df1['Building_walls'].apply(tuple)


In [101]:
final.to_excel(r"C:\Users\Kalpesh\Downloads\Jayesh Testing\analysis_results_eff\Tower2_final_result.xlsx",index=False)